In [1]:
!pip install mysql-connector-python

# Final Project Database Population
## populates all databases

In [2]:
from mysql.connector import connect, Error
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

conn = None

def connect_to_db():
    try:
        global conn
        conn = connect(
            host='localhost',
            user='root',
            password='password',
            database='final'
        )
        print("Connection successful!")
        return conn
    except Error as e:
        print(f"Error: {e}")


In [3]:
car_dekho_path = '../../data/cleaned/car_dekho.csv'
clean_car_sales_data_path = '../../data/cleaned/clean_car_sales_data.csv'
used_cars_data_path = '../../data/cleaned/used_cars_data.csv'
data_path = '../../data/cleaned/enterprise_station.csv'

enterprise_station_df = pd.read_csv(data_path)
car_dekho_data = pd.read_csv(car_dekho_path)
clean_car_sales_data = pd.read_csv(clean_car_sales_data_path)
used_cars_data = pd.read_csv(used_cars_data_path)

car_rental_path = '../../data/cleaned/car_rental_sample.csv'
clean_car_sales_data_path = '../../data/cleaned/clean_car_sales_data.csv'

rental_data = pd.read_csv(car_rental_path)
clean_car_sales_data = pd.read_csv(clean_car_sales_data_path)

sales_data_path = '../../data/cleaned/clean_car_sales_data.csv'
sales_data = pd.read_csv(sales_data_path)

FileNotFoundError: [Errno 2] No such file or directory: '../../data/cleaned/enterprise_station.csv'

## Create / Reset Schema

In [ ]:
def create_schema(conn):
    """
    Create the 'final' schema and all required tables.
    """
    schema_sql = """
    DROP SCHEMA IF EXISTS final;
    CREATE SCHEMA final;
    USE final;

    CREATE TABLE make (
        make_id INT AUTO_INCREMENT PRIMARY KEY,
        make_name VARCHAR(255)
    );

    CREATE TABLE vehicle (
        vehicle_id INT AUTO_INCREMENT PRIMARY KEY,
        make_id INT,
        model VARCHAR(255),
        year INT,
        price INT,
        commission_rate DOUBLE,
        FOREIGN KEY (make_id) REFERENCES make(make_id)
    );

    CREATE TABLE store (
        store_id INT PRIMARY KEY,
        brand VARCHAR(255),
        location_name VARCHAR(255),
        location_number VARCHAR(255),
        location_type VARCHAR(255),
        address VARCHAR(255),
        country VARCHAR(255),
        city VARCHAR(255),
        state VARCHAR(255),
        postal_code VARCHAR(255),
        latitude DOUBLE,
        longitude DOUBLE
    );

    CREATE TABLE staff (
        staff_id INT AUTO_INCREMENT PRIMARY KEY,
        name VARCHAR(255),
        store_id INT,
        hire_date DATE,
        FOREIGN KEY (store_id) REFERENCES store(store_id)
    );

    CREATE TABLE client (
        client_id INT AUTO_INCREMENT PRIMARY KEY,
        name VARCHAR(255),
        salary INT,
        purchased TINYINT(1)
    );

    CREATE TABLE rental (
        rental_id INT AUTO_INCREMENT PRIMARY KEY,
        vehicle_id INT,
        client_id INT,
        rental_rate INT,
        rental_length INT,
        start_date DATE,
        start_time TIME,
        return_date DATE,
        return_time TIME,
        FOREIGN KEY (vehicle_id) REFERENCES vehicle(vehicle_id),
        FOREIGN KEY (client_id) REFERENCES client(client_id)
    );

    CREATE TABLE sale (
        sale_id INT AUTO_INCREMENT PRIMARY KEY,
        vehicle_id INT,
        client_id INT,
        staff_id INT,
        sale_price INT,
        commission_rate DOUBLE,
        sale_date DATE,
        FOREIGN KEY (vehicle_id) REFERENCES vehicle(vehicle_id),
        FOREIGN KEY (client_id) REFERENCES client(client_id),
        FOREIGN KEY (staff_id) REFERENCES staff(staff_id)
    );

    CREATE TABLE commission (
        commission_id INT AUTO_INCREMENT PRIMARY KEY,
        staff_id INT,
        sale_id INT,
        amount DOUBLE,
        FOREIGN KEY (staff_id) REFERENCES staff(staff_id),
        FOREIGN KEY (sale_id) REFERENCES sale(sale_id)
    );

    CREATE TABLE vehicle_performance (
        performance_id INT AUTO_INCREMENT PRIMARY KEY,
        vehicle_id INT,
        rating DOUBLE,
        trips_taken INT,
        review_count INT,
        FOREIGN KEY (vehicle_id) REFERENCES vehicle(vehicle_id)
    );

    CREATE TABLE transaction_history (
        transaction_id INT AUTO_INCREMENT PRIMARY KEY,
        rental_id INT,
        store_id INT,
        payment INT,
        FOREIGN KEY (rental_id) REFERENCES rental(rental_id),
        FOREIGN KEY (store_id) REFERENCES store(store_id)
    );

    CREATE TABLE client_account (
        account_id INT AUTO_INCREMENT PRIMARY KEY,
        client_id INT,
        balance DECIMAL(10, 2),
        created_date DATE,
        status VARCHAR(255),
        FOREIGN KEY (client_id) REFERENCES client(client_id)
    );
    """
    cursor = conn.cursor()
    try:
        # Execute schema creation SQL
        for result in cursor.execute(schema_sql, multi=True):
            if result.with_rows:
                print(f"Rows returned: {result.fetchall()}")
            else:
                print(f"Affected {result.rowcount} rows.")
        print("Schema created successfully.")
    except Error as e:
        print(f"Error creating schema: {e}")
    finally:
        cursor.close()

# Use existing connection
conn = connect_to_db()
if conn:
    try:
        create_schema(conn)
    finally:
        conn.close()
else:
    print("Failed to connect to the database.")


Connection successful!
Affected 11 rows.
Affected 1 rows.
Affected 0 rows.
Affected 0 rows.
Affected 0 rows.
Affected 0 rows.
Affected 0 rows.
Affected 0 rows.
Affected 0 rows.
Affected 0 rows.
Affected 0 rows.
Affected 0 rows.
Affected 0 rows.
Affected 0 rows.
Schema created successfully.


## Populate vehicles

In [ ]:
def handle_missing_makes(vehicle_data, conn):
    """
    Handles missing make_name entries by adding them to the make table in bulk.
    """
    try:
        # Extract all unique makes from the vehicle data
        unique_makes = vehicle_data['make_name'].dropna().unique()

        # Fetch existing makes from the database
        existing_makes = pd.read_sql("SELECT make_name FROM make", conn)['make_name'].tolist()

        # Identify missing makes
        missing_makes = [make for make in unique_makes if make not in existing_makes]

        if missing_makes:
            print(f"Adding missing makes: {missing_makes}")
            
            cursor = conn.cursor()
            insert_query = "INSERT INTO make (make_name) VALUES (%s)"
            
            # Insert each missing make
            cursor.executemany(insert_query, [(make,) for make in missing_makes])
            conn.commit()
            cursor.close()
            print(f"Added {len(missing_makes)} missing makes.")
        else:
            print("No missing makes to add.")
    except Error as e:
        print(f"Error in handle_missing_makes: {e}")

def prepare_vehicle_data(vehicle_data, conn, row_limit=20000):
    """
    Prepares the vehicle data and resolves missing make_name issues.
    Limits the processed data to the specified row limit.
    """
    # Step 1: Handle missing makes (process all makes, regardless of row limit)
    handle_missing_makes(vehicle_data, conn)

    # Step 2: Reload the updated make table
    make_table = pd.read_sql("SELECT * FROM make", conn)

    # Step 3: Merge make_id into vehicle_data
    vehicle_data = vehicle_data.merge(make_table, on='make_name', how='left')

    # Step 4: Filter out rows with missing make_id
    vehicle_data = vehicle_data[vehicle_data['make_id'].notna()]

    # Step 5: Reapply row limit
    vehicle_data = vehicle_data.head(row_limit)

    # Step 6: Truncate long model values to match schema constraints
    vehicle_data['model'] = vehicle_data['model'].str.slice(0, 50)

    # Step 7: Handle missing or invalid `commission_rate` values
    vehicle_data['commission_rate'] = vehicle_data['commission_rate'].fillna(0.0)  # Replace NaN with 0.0

    return vehicle_data[['make_id', 'model', 'year', 'price', 'commission_rate']]


# Main script
conn = connect_to_db()

if conn:
    try:
        # Combine all datasets into a single DataFrame
        car_dekho_vehicles = car_dekho_data[['name', 'year', 'price']].rename(columns={
            'name': 'model', 'year': 'year', 'price': 'price'
        })
        car_dekho_vehicles['make_name'] = car_dekho_vehicles['model'].str.split().str[0]
        car_dekho_vehicles['commission_rate'] = None  # No commission rate in this dataset

        used_cars_vehicles = used_cars_data[['brand', 'model', 'model_year', 'price']].rename(columns={
            'brand': 'make_name', 'model': 'model', 'model_year': 'year', 'price': 'price'
        })
        used_cars_vehicles['commission_rate'] = None  # No commission rate in this dataset

        sales_vehicles = clean_car_sales_data[['Car Make', 'Car Model', 'Car Year', 'Sale Price', 'Commission Rate']].rename(columns={
            'Car Make': 'make_name', 'Car Model': 'model', 'Car Year': 'year', 'Sale Price': 'price', 'Commission Rate': 'commission_rate'
        })

        # Combine all datasets
        combined_vehicles = pd.concat([sales_vehicles, car_dekho_vehicles, used_cars_vehicles], ignore_index=True)

        # Prepare and clean vehicle data with row limit applied
        vehicle_data = prepare_vehicle_data(combined_vehicles, conn, row_limit=20000)

        # Populate the vehicle table
        populate_vehicle_table(vehicle_data, conn)

    except Error as e:
        print(f"Error: {e}")
    finally:
        conn.close()
else:
    print("Failed to connect to the database.")


Connection successful!


/var/folders/zx/lp1b9ndj1kb78j95mcrtw_w80000gp/T/ipykernel_32441/312344124.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  existing_makes = pd.read_sql("SELECT make_name FROM make", conn)['make_name'].tolist()
/var/folders/zx/lp1b9ndj1kb78j95mcrtw_w80000gp/T/ipykernel_32441/312344124.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  make_table = pd.read_sql("SELECT * FROM make", conn)


Adding missing makes: ['Nissan', 'Ford', 'Honda', 'Toyota', 'Chevrolet', 'Maruti', 'Hyundai', 'Datsun', 'Tata', 'Jaguar', 'Mercedes-Benz', 'Audi', 'Skoda', 'Jeep', 'BMW', 'Mahindra', 'Renault', 'Fiat', 'Volkswagen', 'Volvo', 'Mitsubishi', 'Land', 'Daewoo', 'MG', 'Force', 'Isuzu', 'OpelCorsa', 'Ambassador', 'Kia', 'Lexus', 'INFINITI', 'Acura', 'Tesla', 'Aston', 'Lincoln', 'Dodge', 'Genesis', 'Bentley', 'Lucid', 'MINI', 'Porsche', 'Hummer', 'Chrysler', 'Cadillac', 'Lamborghini', 'Maserati', 'Subaru', 'Rivian', 'GMC', 'RAM', 'Alfa', 'Ferrari', 'Scion', 'Mazda', 'Saturn', 'Polestar', 'Rolls-Royce', 'McLaren', 'Buick', 'Lotus', 'Pontiac', 'FIAT', 'Karma', 'Saab', 'Mercury', 'Plymouth', 'smart', 'Maybach', 'Suzuki']
Added 69 missing makes.
Data inserted into `vehicle` table successfully!


## Populate store

In [ ]:
store_data = enterprise_station_df[
    ['tid', 'brand', 'loc_name', 'loc_number', 'loc_type', 'address_1',
     'country', 'city', 'state', 'postal_code', 'latitude', 'longitude']
].rename(columns={
    'tid': 'store_id',
    'loc_name': 'location_name',
    'loc_number': 'location_number',
    'loc_type': 'location_type',
    'address_1': 'address'
}).replace({np.nan: None})

print("Prepared data:")

conn = connect_to_db()

if conn:
    try:
        cursor = conn.cursor()
        
        insert_query = """
        INSERT INTO store (
            store_id, brand, location_name, location_number, location_type, 
            address, country, city, state, postal_code, latitude, longitude
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        
        data_tuples = list(store_data.itertuples(index=False, name=None))
        cursor.executemany(insert_query, data_tuples)
        
        conn.commit()
        print("Data inserted successfully into the `store` table!")
    
    except Error as e:
        print(f"Error: {e}")
    
    finally:
        cursor.close()
        conn.close()
else:
    print("Failed to connect to the database.")

Prepared data:
Connection successful!
Data inserted successfully into the `store` table!


## Populate client, staff, sale, & commissions

In [ ]:
LIMIT = 10000
def prepare_client_data(sales_data, conn, batch_size=1000, row_limit=LIMIT):
    cursor = None
    try:
        sales_data = sales_data.iloc[:row_limit]
        
        client_data = sales_data[['Customer Name', 'Sale Price']].drop_duplicates()
        client_data = client_data.rename(columns={'Customer Name': 'name'})
        client_data['age'] = None 
        client_data['salary'] = client_data['Sale Price'] * 3.5
        client_data['purchased'] = 1 
        client_data = client_data[['name', 'salary', 'purchased']]

        cursor = conn.cursor()
        insert_query = """
        INSERT INTO client (name, salary, purchased)
        VALUES (%s, %s, %s)
        """
        for i in range(0, len(client_data), batch_size):
            batch = client_data.iloc[i:i+batch_size].to_records(index=False).tolist()
            cursor.executemany(insert_query, batch)
            conn.commit()
    except Error as e:
        print(f"Error preparing client data: {e}")
    finally:
        if cursor:
            cursor.close()


def prepare_staff_data(sales_data, conn, row_limit=LIMIT):
    cursor = None
    try:
        sales_data = sales_data.iloc[:row_limit]

        staff_data = sales_data[['Salesperson']].drop_duplicates()
        staff_data = staff_data.rename(columns={'Salesperson': 'name'})

        store_ids = pd.read_sql("SELECT store_id FROM store", conn)['store_id'].tolist()
        if not store_ids:
            raise ValueError("No store_ids found in the `store` table.")

        staff_data['store_id'] = [random.choice(store_ids) for _ in range(len(staff_data))]

        staff_data['hire_date'] = pd.Timestamp.now().strftime('%Y-%m-%d')

        cursor = conn.cursor()
        insert_query = """
        INSERT INTO staff (name, store_id, hire_date)
        VALUES (%s, %s, %s)
        """
        cursor.executemany(insert_query, staff_data.to_records(index=False).tolist())
        conn.commit()
        print(f"Data inserted into `staff` table successfully! {len(staff_data)} records.")
    except Error as e:
        print(f"Error preparing staff data: {e}")
    except ValueError as ve:
        print(f"Error: {ve}")
    finally:
        if cursor:
            cursor.close()


def prepare_sale_data(sales_data, conn, batch_size=1000, row_limit=LIMIT):
    """
    Prepares and populates the `sale` table, including the commission_rate column.
    """
    cursor = None
    try:
        sales_data = sales_data.iloc[:row_limit]

        sale_data = sales_data[['Customer Name', 'Salesperson', 'Car Make', 'Sale Price', 'Commission Rate']].rename(
            columns={
                'Customer Name': 'name',
                'Salesperson': 'staff_name',
                'Car Make': 'make_name',
                'Sale Price': 'sale_price',
                'Commission Rate': 'commission_rate'
            }
        ).drop_duplicates()

        client_ids = pd.read_sql("SELECT client_id, name FROM client", conn)
        staff_ids = pd.read_sql("SELECT staff_id, name AS staff_name FROM staff", conn)
        make_ids = pd.read_sql("SELECT make_id, make_name FROM make", conn)
        vehicle_ids = pd.read_sql("SELECT vehicle_id, make_id FROM vehicle", conn)

        sale_data = sale_data.merge(client_ids, on='name', how='left')
        sale_data = sale_data.merge(staff_ids, on='staff_name', how='left')
        sale_data = sale_data.merge(make_ids, on='make_name', how='left')
        sale_data = sale_data.merge(vehicle_ids, on='make_id', how='left')

        sale_data = sale_data.dropna(subset=['vehicle_id', 'client_id', 'staff_id'])

        sale_data = sale_data.head(row_limit)

        sale_data = sale_data[['vehicle_id', 'client_id', 'staff_id', 'sale_price', 'commission_rate']]
        sale_data = sale_data.astype({
            'vehicle_id': 'int',
            'client_id': 'int',
            'staff_id': 'int',
            'commission_rate': 'float'
        })

        cursor = conn.cursor()
        insert_query = """
        INSERT INTO sale (vehicle_id, client_id, staff_id, sale_price, commission_rate, sale_date)
        VALUES (%s, %s, %s, %s, %s, CURDATE())
        """
        for start in range(0, len(sale_data), batch_size):
            batch = sale_data.iloc[start:start + batch_size].to_records(index=False).tolist()
            cursor.executemany(insert_query, batch)
            conn.commit()
            print(f"Inserted {len(batch)} records (Batch {start // batch_size + 1}).")

    except Error as e:
        print(f"Error preparing sale data: {e}")
    finally:
        if cursor:
            cursor.close()

def prepare_commission_data(conn, row_limit=LIMIT):
    """
    Prepares and populates the `commission` table using sale data.
    """
    cursor = None
    try:
        sale_data = pd.read_sql(f"""
            SELECT sale_id, staff_id, sale_price, commission_rate 
            FROM sale 
            LIMIT {row_limit}
        """, conn)

        sale_data['amount'] = sale_data['sale_price'] * sale_data['commission_rate']

        cursor = conn.cursor()
        insert_query = """
        INSERT INTO commission (sale_id, staff_id, amount)
        VALUES (%s, %s, %s)
        """
        commission_records = sale_data[['sale_id', 'staff_id', 'amount']].to_records(index=False).tolist()

        cursor.executemany(insert_query, commission_records)
        conn.commit()
        print(f"Data inserted into `commission` table successfully! {len(commission_records)} records.")
    except Error as e:
        print(f"Error preparing commission data: {e}")
    finally:
        if cursor:
            cursor.close()

conn = connect_to_db()
if conn:
    try:
        prepare_client_data(sales_data, conn)
        prepare_staff_data(sales_data, conn)
        prepare_sale_data(sales_data, conn)
        prepare_commission_data(conn)
    finally:
        conn.close()
else:
    print("Failed to connect to the database.")


Connection successful!
Data inserted into `staff` table successfully! 9354 records.


/var/folders/zx/lp1b9ndj1kb78j95mcrtw_w80000gp/T/ipykernel_32441/3077918814.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  store_ids = pd.read_sql("SELECT store_id FROM store", conn)['store_id'].tolist()
/var/folders/zx/lp1b9ndj1kb78j95mcrtw_w80000gp/T/ipykernel_32441/3077918814.py:81: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  client_ids = pd.read_sql("SELECT client_id, name FROM client", conn)
/var/folders/zx/lp1b9ndj1kb78j95mcrtw_w80000gp/T/ipykernel_32441/3077918814.py:82: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlc

Inserted 1000 records (Batch 1).
Inserted 1000 records (Batch 2).
Inserted 1000 records (Batch 3).
Inserted 1000 records (Batch 4).
Inserted 1000 records (Batch 5).
Inserted 1000 records (Batch 6).
Inserted 1000 records (Batch 7).
Inserted 1000 records (Batch 8).
Inserted 1000 records (Batch 9).
Inserted 1000 records (Batch 10).
Data inserted into `commission` table successfully! 10000 records.


/var/folders/zx/lp1b9ndj1kb78j95mcrtw_w80000gp/T/ipykernel_32441/3077918814.py:126: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sale_data = pd.read_sql(f"""


## Populate rental, vehicle_performance

In [ ]:
def populate_vehicle_performance_table(rental_data, conn):
    """
    Populate the vehicle_performance table based on rental reviews.
    If a vehicle_id is missing, assign a random vehicle_id from the vehicle table.
    """
    cursor = conn.cursor()
    try:
        vehicle_ids = pd.read_sql("SELECT vehicle_id FROM vehicle", conn)['vehicle_id'].tolist()

        vehicle_map = get_vehicle_id_from_db(conn)
        rental_data['vehicle_id'] = rental_data['product_name'].str.strip().str.lower().map(vehicle_map)

        missing_vehicle_ids = rental_data['vehicle_id'].isna().sum()
        rental_data['vehicle_id'] = rental_data['vehicle_id'].apply(
            lambda x: x if pd.notna(x) else random.choice(vehicle_ids)
        )
        print(f"Missing vehicle_id mappings: {missing_vehicle_ids} replaced with random IDs.")

        performance_data = rental_data.groupby('vehicle_id').agg(
            rating=('average', 'mean'),
            trips_taken=('rental_length', 'sum'),
            review_count=('no_of_ratings', 'sum')
        ).reset_index()

        performance_data = performance_data.fillna({
            'rating': 0.0,
            'trips_taken': 0,
            'review_count': 0
        })

        performance_records = performance_data[['vehicle_id', 'rating', 'trips_taken', 'review_count']].values.tolist()

        print(f"Number of performance records to insert: {len(performance_records)}")

        insert_query = """
        INSERT INTO vehicle_performance (vehicle_id, rating, trips_taken, review_count)
        VALUES (%s, %s, %s, %s)
        """
        cursor.executemany(insert_query, performance_records)
        conn.commit()
        print(f"Data inserted into `vehicle_performance` table successfully! {len(performance_records)} rows.")
    except Error as e:
        print(f"Error inserting into vehicle_performance table: {e}")
    finally:
        cursor.close()


def populate_rental_table(rental_data, conn):
    """
    Populate the rental table from the rental data.
    """
    cursor = conn.cursor()
    try:
        vehicle_map = get_vehicle_id_from_db(conn)
        client_map = get_client_id_from_db(conn)

        vehicle_map = {k.strip().lower(): v for k, v in vehicle_map.items()}
        client_map = {k.strip().lower(): v for k, v in client_map.items()}

        rental_data['vehicle_id'] = rental_data['product_name'].str.strip().str.lower().map(vehicle_map)
        rental_data['client_id'] = rental_data['airport'].str.strip().str.lower().map(client_map)

        unmapped_products = rental_data[rental_data['vehicle_id'].isna()]['product_name'].unique()
        unmapped_airports = rental_data[rental_data['client_id'].isna()]['airport'].unique()
        print(f"Unmapped product_names: {unmapped_products}")
        print(f"Unmapped airports: {unmapped_airports}")

        rental_data['vehicle_id'] = rental_data['vehicle_id'].fillna(random.choice(list(vehicle_map.values())))
        rental_data['client_id'] = rental_data['client_id'].fillna(random.choice(list(client_map.values())))

        rental_records = rental_data[[
            'vehicle_id', 'client_id', 'price', 'rental_length',
            'start_date', 'start_time', 'return_date', 'return_time'
        ]].values.tolist()

        print(f"Number of rental records to insert: {len(rental_records)}")

        insert_query = """
        INSERT INTO rental (vehicle_id, client_id, rental_rate, rental_length, start_date, start_time, return_date, return_time)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """
        cursor.executemany(insert_query, rental_records)
        conn.commit()
        print(f"Data inserted into `rental` table successfully! {len(rental_records)} rows.")
    except Error as e:
        print(f"Error inserting into rental table: {e}")
    finally:
        cursor.close()

def get_vehicle_id_from_db(conn):
    query = "SELECT vehicle_id, model FROM vehicle"
    vehicle_map = pd.read_sql(query, conn).set_index('model')['vehicle_id'].to_dict()
    return vehicle_map

def get_client_id_from_db(conn):
    query = "SELECT client_id, name FROM client"
    client_map = pd.read_sql(query, conn).set_index('name')['client_id'].to_dict()
    return client_map

conn = connect_to_db()
if conn:
    try:
        print(f"Rental data columns: {rental_data.columns}")

        populate_rental_table(rental_data, conn)

        cursor = conn.cursor()
        cursor.execute("SELECT COUNT(*) FROM rental")
        rental_count = cursor.fetchone()[0]
        print(f"Rows in `rental` table after insertion: {rental_count}")
        cursor.close()

        populate_vehicle_performance_table(rental_data, conn)
    finally:
        conn.close()
else:
    print("Failed to connect to the database.")


/var/folders/zx/lp1b9ndj1kb78j95mcrtw_w80000gp/T/ipykernel_32441/3416151994.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vehicle_map = pd.read_sql(query, conn).set_index('model')['vehicle_id'].to_dict()
/var/folders/zx/lp1b9ndj1kb78j95mcrtw_w80000gp/T/ipykernel_32441/3416151994.py:97: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  client_map = pd.read_sql(query, conn).set_index('name')['client_id'].to_dict()


Connection successful!
Rental data columns: Index(['airport', 'airport_iata', 'country', 'city', 'rental_length',
       'start_date', 'start_time', 'return_date', 'return_time', 'date_offset',
       'deposit_price', 'drive_away_price', 'price', 'currency',
       'product_name', 'product_id', 'airbags', 'aircon', 'free_cancellation',
       'doors', 'group', 'seats', 'transmission', 'mileage', 'supplier_name',
       'supplier_address', 'supplier_loction_type', 'average', 'average_text',
       'cleanliness', 'condition', 'dropoff_time', 'efficiency', 'location',
       'pickup_time', 'value_for_money', 'no_of_ratings', 'RunDate',
       'setup_prams', 'tid'],
      dtype='object')
Unmapped product_names: ['Fiat 500' 'Kia Rio' 'Kia Stonic' 'Seat Leon' 'Honda CR-V'
 'Volkswagen Golf' 'Nissan Qashqai' 'Nissan Qashqai   ' 'Ford Mondeo'
 'Kia Sportage' 'Vauxhall Corsa' 'Polestar 2' 'Vauxhall Crossland X'
 'Peugeot 2008' 'Ford Fiesta' 'Ford Focus' 'Volkswagen Passat'
 'Citroen C3 Aircross

/var/folders/zx/lp1b9ndj1kb78j95mcrtw_w80000gp/T/ipykernel_32441/3416151994.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vehicle_ids = pd.read_sql("SELECT vehicle_id FROM vehicle", conn)['vehicle_id'].tolist()
/var/folders/zx/lp1b9ndj1kb78j95mcrtw_w80000gp/T/ipykernel_32441/3416151994.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vehicle_map = pd.read_sql(query, conn).set_index('model')['vehicle_id'].to_dict()


## Populate client_account

In [ ]:
def populate_client_accounts(conn):
    """
    Populate the client_account table with made-up data for all client IDs from the client table.
    """
    cursor = conn.cursor()
    try:
        client_ids = pd.read_sql("SELECT client_id FROM client", conn)['client_id'].tolist()

        client_accounts = []
        for client_id in client_ids:
            balance = round(random.uniform(0, 5000), 2)
            created_date = datetime.now() - timedelta(days=random.randint(0, 365 * 5))
            status = random.choice(['Active', 'Inactive', 'Suspended'])
            client_accounts.append((client_id, balance, created_date.date(), status))

        print(f"Number of client accounts to insert: {len(client_accounts)}")
        print(f"Sample data: {client_accounts[:5]}")

        insert_query = """
        INSERT INTO client_account (client_id, balance, created_date, status)
        VALUES (%s, %s, %s, %s)
        """
        cursor.executemany(insert_query, client_accounts)
        conn.commit()
        print(f"Data inserted into `client_account` table successfully! {len(client_accounts)} rows.")
    except Error as e:
        print(f"Error inserting into client_account table: {e}")
    finally:
        cursor.close()

conn = connect_to_db()
if conn:
    try:
        populate_client_accounts(conn)
    finally:
        conn.close()
else:
    print("Failed to connect to the database.")



Connection successful!
Number of client accounts to insert: 10000
Sample data: [(1, 3439.92, datetime.date(2022, 4, 18), 'Active'), (2, 4578.42, datetime.date(2022, 9, 24), 'Active'), (3, 2946.19, datetime.date(2022, 1, 19), 'Active'), (4, 4952.3, datetime.date(2022, 9, 6), 'Active'), (5, 2655.94, datetime.date(2023, 8, 17), 'Active')]


/var/folders/zx/lp1b9ndj1kb78j95mcrtw_w80000gp/T/ipykernel_32441/2732936351.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  client_ids = pd.read_sql("SELECT client_id FROM client", conn)['client_id'].tolist()


Data inserted into `client_account` table successfully! 10000 rows.


## Check that all tables have been populated

In [ ]:
def check_table_populations(conn):
    tables = [
        "make", "vehicle", "store", "staff", "client", "rental", 
        "sale", "commission", "vehicle_performance", "client_account"
    ]
    
    cursor = conn.cursor()
    try:
        cursor.execute("USE final;")
        
        table_counts = {}
        for table in tables:
            cursor.execute(f"SELECT COUNT(*) FROM {table};")
            count = cursor.fetchone()[0]
            table_counts[table] = count
        
        print("Table population check:")
        for table, count in table_counts.items():
            status = "Populated" if count > 0 else "Empty"
            print(f"Table `{table}`: {count} rows ({status})")
        
        all_populated = all(count > 0 for count in table_counts.values())
        if all_populated:
            print("\nAll tables are populated!")
        else:
            print("\nSome tables are empty. Check the data loading process.")
    
    except Error as e:
        print(f"Error checking table populations: {e}")
    finally:
        cursor.close()

conn = connect_to_db()
if conn:
    try:
        check_table_populations(conn)
    finally:
        conn.close()
else:
    print("Failed to connect to the database.")


Connection successful!
Table population check:
Table `make`: 69 rows (Populated)
Table `vehicle`: 20000 rows (Populated)
Table `store`: 4177 rows (Populated)
Table `staff`: 9354 rows (Populated)
Table `client`: 10000 rows (Populated)
Table `rental`: 2730 rows (Populated)
Table `sale`: 10000 rows (Populated)
Table `commission`: 10000 rows (Populated)
Table `vehicle_performance`: 2571 rows (Populated)
Table `client_account`: 10000 rows (Populated)

All tables are populated!
